In [24]:
import os
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import multiprocessing as mp

In [40]:
comic = '4240.html'
save_path = '/Users/admin/Desktop/Domekano/'

url_base = 'http://nyaso.com/comic/'
url_index = url_base + comic

UA = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.86 Safari/537.36'
header = {"User-Agent" : UA,}
proxies={
    'http':'127.0.0.1:1087',
    'https':'127.0.0.1:1087'
}



In [41]:
# Get Episodes
r = requests.get(url_index, headers=header, proxies=proxies)
r.encoding = 'utf8'

soup = BeautifulSoup(r.text, 'lxml')
eps = soup.findAll('a', target='new')
ep_list = []
for e in eps:
    ep_list.append({'name':e.text.replace(' ','-'),'link':e['href']})
ep_list = sorted(ep_list, key=lambda x: x['name'])

In [42]:
# Get Images
service_args = [
    '--proxy=127.0.0.1:1087',    # 代理 IP：prot    （eg：192.168.0.28:808）
    '--proxy-type=http',            # 代理类型：http/https
    '--ignore-ssl-errors=true'    # 忽略https错误（可选）
]

driver = webdriver.PhantomJS(executable_path='/usr/local/bin/phantomjs', service_args=service_args)
driver.command_executor._commands['executePhantomScript'] = ('POST', '/session/$sessionId/phantom/execute')
script = 'this.browserLog.push(this.evaluate(function(){var l = [];for (i = 0; i < gallery.data.length; i++){l.push({"title":gallery.data[i].title,"url":"https:"+gallery.data[i].slideUrl})} return l}))'
def get_img_list(url):
    while True: 
        try:
            driver.get(url)
            driver.execute('executePhantomScript', {'script': script, 'args': []})
            return driver.get_log('browser')[0][:-1]
#             break
        except:
            print('Error Loading Image List'+url)
    

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


In [43]:
def save_imgs(imgs, dir_):
    os.system('mkdir '+save_path+dir_+'.ing')
    print('Downloading...'+dir_)
    for each in imgs:
        while True: 
            try:
                with open(save_path+dir_+'.ing/'+each['title'].replace(' ','-')+'.jpg','wb') as f:
                    f.write(requests.get(each['url'], proxies=proxies, headers=header).content)
#                     f.write(requests.get(each['url']).content, headers=header)
                break
            except:
                print('Err: '+each['title'])
    os.system('mv '+save_path+dir_+'.ing '+save_path+dir_)

In [44]:
eb = ep_list

In [45]:
ep_list = []
for i in li:
    ep_list.append(eb[i])
ep_list.append(eb[-1])
ep_list

[{'name': '069话-18P', 'link': '424000692018001.html'},
 {'name': '077话-17P', 'link': '424000772017001.html'},
 {'name': '182话-18P', 'link': '424001822018001.html'},
 {'name': '177话-18P', 'link': '424001772018001.html'},
 {'name': '176话-18P', 'link': '424001762018001.html'},
 {'name': '183话-18P', 'link': '424001832018001.html'},
 {'name': '078话-21P', 'link': '424000782021001.html'},
 {'name': '075话-18P', 'link': '424000752018001.html'},
 {'name': '188话-18P', 'link': '424001882018001.html'},
 {'name': '073话-22P', 'link': '424000732022001.html'},
 {'name': '185话-18P', 'link': '424001852018001.html'},
 {'name': '074话-18P', 'link': '424000742018001.html'},
 {'name': '186话-18P', 'link': '424001862018001.html'},
 {'name': '174话-16P', 'link': '424001742016001.html'},
 {'name': '072话-17P', 'link': '424000722017001.html'},
 {'name': '168话-18P', 'link': '424001682018001.html'},
 {'name': '187话-18P', 'link': '424001872018001.html'},
 {'name': '076话-18P', 'link': '424000762018001.html'},
 {'name': 

In [49]:
pool = mp.Pool( processes=10 )

for each in ep_list[:]:
    pool.apply_async( save_imgs, [get_img_list(url_base+each['link']), each['name']] )

Downloading...069话-18P
Downloading...077话-17P
Downloading...182话-18P
Error Loading Image Listhttp://nyaso.com/comic/424001772018001.html
Downloading...177话-18P
Error Loading Image Listhttp://nyaso.com/comic/424001762018001.html
Error Loading Image Listhttp://nyaso.com/comic/424001762018001.html
Downloading...176话-18P
Downloading...183话-18P
Downloading...078话-21P
Error Loading Image Listhttp://nyaso.com/comic/424000752018001.html
Error Loading Image Listhttp://nyaso.com/comic/424000752018001.html
Error Loading Image Listhttp://nyaso.com/comic/424000752018001.html
Error Loading Image Listhttp://nyaso.com/comic/424000752018001.html
Downloading...075话-18P
Downloading...188话-18P
Downloading...073话-22P
Err: 第75话 P9
Downloading...185话-18P
Downloading...074话-18P
Downloading...186话-18P
Error Loading Image Listhttp://nyaso.com/comic/424001742016001.html
Downloading...174话-16P
Downloading...072话-17P
Error Loading Image Listhttp://nyaso.com/comic/424001682018001.html
Error Loading Image Listhttp:/

In [56]:
import sys
import os
from reportlab.lib.pagesizes import A4, landscape
from reportlab.pdfgen import canvas
from PIL import Image

os.chdir('/Users/aeonni/Desktop/Domekano/')

In [50]:
# 检查文件错误
li = []
d = []
for each in os.listdir('/Users/aeonni/Desktop/Domekano/'):
    if each not in [ '.DS_Store', 'PDF']:
        for file in os.listdir('/Users/aeonni/Desktop/Domekano/'+each):
            if file != '.DS_Store':
                if int(file[file.find('第')+1:file.find('话')]) != int(each[:3]):
                    print(each, file[file.find('第')+1:file.find('话')] )
                    li.append(int(each[:3])-1)
                    d.append(each)
                    break

In [59]:
def pic2pdf(path, name, savepath = './PDF/'):
    fs = os.listdir(path)
    # 删除非支持文件，并按照文件名排序
    for each in fs: 
        if ('.jpg' or '.png' or '.jpeg') not in each:
            fs.remove(each)
#     fs = sorted(fs, key=lambda s: int(s[s.find('-P')+2:-4]))
    fs = sorted(fs)
#     print(fs)
    # 寻找适合的画布大小
    # w = 0
    # h = 0
    # for each in fs:
    #     img = Image.open(path+each)
    #     iw, ih = img.size
    #     w = max(w,iw)
    #     h = max(h,ih)
    # 确定PDF的名称，并生成PDF
    
    f_pdf = savepath + name+'.pdf'
    c = canvas.Canvas(f_pdf)
    for each in fs:
        # (w, h) = landscape(A4)
        img = Image.open(path+each)
        c.setPageSize(img.size)
        # print(c._pagesize)
        c.drawImage(path+each, 0, 0, img.size[0], img.size[1])
        c.showPage()
    c.save()
    return f_pdf

In [39]:
for each in os.listdir():
    print(each)
    try:
        pic2pdf(each+'/',each)
    except:
        print('ERROR: ',each)

.DS_Store
ERROR:  .DS_Store
[御の萌漫屋][流石景]家有女友 第198.5话


In [33]:
pic2pdf('065话-18P/','065话-18P')

'./PDF/065话-18P.pdf'

In [1]:
import sys
import os

In [60]:
for each in ep_list:
     pic2pdf(each['name']+'/',each['name'])

In [48]:
for each in d:
    os.system('rm -r '+save_path+each)